In [1]:
# Read CSV file from blob storage
storage_account_name = 'str1wk6'
storage_account_access_key = 'OnTwDpJa14kT8Xi9rY36r9kKeUqNXJdCm1D4RLl9LgUUGBFdmKjQoWhpn2dljkfDAhuDXEprXyB7Ld0eFW+upQ=='
containerName = 'cnt1'
fileName = 'google_stock_test_data.csv'
# Get data of test csv file from blob storage
spark.conf.set("fs.azure.account.key."+storage_account_name+".blob.core.windows.net",storage_account_access_key)
df = spark.read.format('csv').options(header='true',inferSchema='true').load('wasbs://cnt1@str1wk6.blob.core.windows.net/google_stock_test_data.csv')

In [2]:
x = df.select('High','Low').toPandas()
y = df.select('Open').toPandas()

In [3]:
# Download file from blob storage
from azure.storage.blob import *
import pickle
import os

if not os.path.exists('/opt/pkl_folder'):
  os.mkdir('/opt/pkl_folder')

blob_storage = BlockBlobService(account_name=storage_account_name, account_key=storage_account_access_key)
blob_storage.get_blob_to_path(containerName, 'LinearRegression.pkl', '/opt/pkl_folder/LinearRegression.pkl')

pkl_file = open('/opt/pkl_folder/LinearRegression.pkl', 'rb')
regressor = pickle.load(pkl_file)

In [4]:
from sklearn.metrics import r2_score
y_pred = regressor.predict(x)
accuracy = r2_score(y, y_pred)
print("Accuracy", accuracy)

In [5]:
import pandas as pd
y_pred_df = pd.DataFrame(y_pred, columns=['y_pred'])
result_df = pd.concat([x, y_pred_df], axis=1)
spark_result_df = spark.createDataFrame(result_df)
spark_result_df.show()

In [6]:
# Save dataframe to cosmosDB
writeConfig = {
 "Endpoint" : "https://predictresultdb.documents.azure.com:443/",
 "Masterkey" : "RC9QHfB7DAbpKkXT8spsuYNnYFBW07mYlpYuZATc4lk9atAWTKpoz3Gzzhe3M6cbHhFjEmppEExqDeBmP8EeVA==",
 "Database" : "ToDoList",
 "Collection" : "Items",
 "Upsert" : "true"
}

spark_result_df.write.format("com.microsoft.azure.cosmosdb.spark").mode('overwrite').options(**writeConfig).save()


In [7]:
# Read table from cosmos DB
# readConfig = {
#   "Endpoint" : "https://predictresultdb.documents.azure.com:443/",
#  "Masterkey" : "RC9QHfB7DAbpKkXT8spsuYNnYFBW07mYlpYuZATc4lk9atAWTKpoz3Gzzhe3M6cbHhFjEmppEExqDeBmP8EeVA==",
#  "Database" : "ToDoList",
#  "Collection" : "Items",
#   "query_custom" : "select c.High, c.Low, c.y_pred from c"
# }

# # Connect via azure-cosmosdb-spark to create Spark DataFrame
# df = spark.read.format("com.microsoft.azure.cosmosdb.spark").options(**readConfig).load()
# df.show()